In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import os
import argparse
from inflation import BBI

import numpy as np 
from experiments.cifar import cifar
from experiments.PDE_PoissonD import PDE_PoissonD

from run_experiment_hyperopt import *

from hyperopt import hp, tpe, Trials, fmin
import json

!mkdir -p results
experiment = "cifar"

tune_epochs = 3  #number of epochs used during the tuning
n_trials = 50    #number of evaluations for the tuning, for each optimizer
check_epochs = 150 # number of epochs for checking the performance after the tuning

# for the general Poisson experiment, choose here the problem number
problem_number = None

seed = 42

#fixed BBI parameters
threshold_BBI = 2000
threshold0_BBI = 100
consEn_BBI = True
nFixedBounces_BBI = 100
deltaEn = 0.0

scanning_pars = {
    'tune_epochs': tune_epochs,
    'n_trials': n_trials,
    'check_epochs': check_epochs,
    'seed': seed,
    'sgd': {'stepsize': [0.001, 0.2], 'rho': [0.8,1.0]},
    'problem': problem_number,
    'BBI' : {'stepsize': [0.001, 0.2]}  ,
    'comments': 'test Experiment. Fixed BBI pars:\n'+'\nthreshold_BBI: '+str(threshold_BBI)+
    '\nthreshold0_BBI: '+str(threshold0_BBI)+
    '\nconsEN_BBI: '+str(consEn_BBI)+
    '\nnFixedBounces_BBI: '+str(nFixedBounces_BBI)+ 
    '\ndeltaEn: '+str(deltaEn)
}

with open('results/scanning-parameters-'+experiment+'.json', 'w') as json_file:
  json.dump(scanning_pars, json_file)

print("I am working on ", experiment)
if torch.cuda.is_available(): print("I am running on gpu")
else: print("I am running on cpu")

I am working on  cifar
I am running on gpu


In [2]:

def run_experiment_sgd(epochs = 2, stepsize = 1e-3, rho = .99):
    param_list = ["main.py", experiment, "--optimizer", "sgd",
                           "--lr", str(stepsize), "--rho", str(rho),
                           "--epochs", str(epochs), "--seed", str(seed),
                           "--progress", "false", "--device", "cuda"]
    if experiment == "PDE_PoissonD": 
        param_list.append("--problem")
        param_list.append(str(problem_number))
    
    return run_experiment(param_list)
    
def run_experiment_BBI(epochs = 2, stepsize = 1e-3, threshold = threshold_BBI, threshold0 = threshold0_BBI, consEn = consEn_BBI, nFixedBounces = nFixedBounces_BBI, deltaEn = deltaEn):
    param_list = ["main.py", experiment, "--optimizer", "BBI",
                           "--lr", str(stepsize),
                           "--epochs", str(epochs),"--seed", str(seed),
                           "--threshold", str(threshold),
                           "--threshold0", str(threshold0),
                           "--nFixedBounces", str(nFixedBounces),
                           "--deltaEn", str(deltaEn),
                           "--consEn", str(consEn),
                           "--progress", "false","--device", "cuda"]
    if experiment == "PDE_PoissonD": 
        param_list.append("--problem")
        param_list.append(str(problem_number))
        
    return run_experiment(param_list)

In [3]:
def hyperopt_tuning(ranges, optimizer, epochs=10, n_trials=5):
        
    def optimizer_func(pars):
        return optimizer(epochs=epochs, **pars)
        
    fspace = {}
    for par, range in ranges.items(): fspace[par] = hp.uniform(par, *range)

    trials = Trials()
    best = fmin(fn=optimizer_func, space=fspace, algo=tpe.suggest, trials=trials, max_evals=n_trials)
    return best

In [ ]:
print("Tuning sgd: ")
best_par_sgd = hyperopt_tuning(scanning_pars['sgd'], run_experiment_sgd, epochs = tune_epochs, n_trials=n_trials)
print("best sgd parameters:", best_par_sgd)

In [ ]:
print("Tuning BBI: ")
best_par_BBI = hyperopt_tuning(scanning_pars['BBI'],run_experiment_BBI, epochs = tune_epochs, n_trials=n_trials)
print("best BBI parameters:", best_par_BBI)

In [ ]:
print("Best parameters")
print("sgd:", best_par_sgd)
print("BBI:", best_par_BBI)

best_pars = {
    'sgd': best_par_sgd,
    'BBI': best_par_BBI
}

with open('results/best-parameters-'+experiment+'.json', 'w') as json_file:
  json.dump(best_pars, json_file)

In [ ]:
print("Running experiment with the best parameters for more epochs...")

print("Running BBI: ")
final_loss_BBI = run_experiment_BBI(epochs=check_epochs, **best_par_BBI)
print(final_loss_BBI)

print("Running sgd: ")
final_loss_sgd = run_experiment_sgd(epochs=check_epochs, **best_par_sgd)
print(final_loss_sgd)
